In [1]:
import os
import sys
import random
import time
from random import seed, randint
import argparse
import platform
from datetime import datetime
import imp
import numpy as np
import fileinput
from itertools import product
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
import seaborn as sns
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib as mpl
# sys.path.insert(0,'..')
# from notebookFunctions import *
# from .. import notebookFunctions
from Bio.PDB.Polypeptide import one_to_three
from Bio.PDB.Polypeptide import three_to_one
from Bio.PDB.PDBParser import PDBParser
from pyCodeLib import *
# from small_script.myFunctions import *
sys.path.insert(0, "/Users/weilu/openmmawsem")
from helperFunctions.myFunctions import *
from collections import defaultdict
%matplotlib inline
# plt.rcParams['figure.figsize'] = (10,6.180)    #golden ratio
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
plt.rcParams['figure.figsize'] = np.array([16.18033, 10])    #golden ratio
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['figure.dpi'] = 100
plt.rcParams.update({'font.size': 22})

In [3]:
def mycp(source, target):
    os.system(f"cp {source} {target}")
    
do = os.system

def getSeq(fileLocation):
    p = PDBParser()
    s = p.get_structure("test", fileLocation)
    seq = ""
    residues = list(s.get_residues())
    for residue in residues:
        res_id = residue.get_id()[0]
        if res_id==' ':
            residue_name = residue.get_resname()
            seq += three_to_one(residue_name)
    return seq

# get chains nad seq
def getChainsAndSeq(fileLocation):
    # fileLocation = "/Users/weilu/Research/examples/optimization/optimization/Structure_Ensemble/1.pdb"
    p = PDBParser()
    pdb = p.get_structure("test", fileLocation)
    residues = list(pdb.get_residues())
    seq = ""
    chains = ""
    for residue in residues:
        res_id = residue.get_id()[0]
        chain = residue.get_full_id()[2]
        if res_id==' ':
            residue_name = residue.get_resname()
            seq += three_to_one(residue_name)
            chains += chain
    return chains, seq

In [4]:
pdbFolderList = glob.glob("/Users/weilu/Dropbox/Structure_Ensemble_*")

In [5]:
len(pdbFolderList)

72

# filter pdbFolderList 

In [ ]:
_all = []
# generate decoys
protein_list = []
for pdbFolder in pdbFolderList:
    pdbName = pdbFolder.split("_")[-1]
    source = pdbFolder + f"/*.pdb"
    p_list = glob.glob(source)
    # print(p_list, source)
    assert len(p_list) == 1
    p = p_list[0]
    chain_seq, seq = getChainsAndSeq(p)
    
    # print(pdbName, i, len(seq), len(chain_seq))
    decoy_list = []
    with open(f"{pdbFolder}/peptide.txt") as f:
        for line in f:
            pep = line.strip()
            pep_len = len(pep)
            # assert len(pep) == 9
            for c in list(set(chain_seq)):
                if chain_seq.count(c) == pep_len:
                    first_c = chain_seq.find(c)
                    a = list(seq)
                    a[first_c:first_c+pep_len] = pep
                    assert len(set(chain_seq[first_c:first_c+pep_len])) == 1
                    decoy = "".join(a)
                    decoy_list.append(decoy)
    _all.append([pdbName, len(decoy_list)])

In [8]:
a = pd.DataFrame(_all, columns=["Name", "Length"])
a.query("Length < 1000").sort_values("Name")

,Name,Length
23,3D39,0
62,3D3V,0
13,3VXM,100
46,3VXR,100
41,3VXS,100
45,3VXU,100
40,5YXN,0
37,5YXU,0
68,MAGEA3,200


In [16]:
pdbList = list(a.query("Length == 1000")["Name"])

In [17]:
len(pdbFolderList)

72

In [18]:
len(pdbList)

63

In [20]:
pre = "/Users/weilu/Research/server/sep_2019/peptide_optimization_trial_8_larger_set"
do(f"mkdir -p {pre}")
do(f"mkdir -p {pre}/database/dompdb")
do(f"mkdir -p {pre}/database/S20_seq")
do(f"mkdir -p {pre}/optimization/decoys/shuffle")
# generate decoys
protein_list = []
for pdbFolder in pdbFolderList:
    pdbName = pdbFolder.split("_")[-1]
    if pdbName not in pdbList:
        continue
    source = pdbFolder + f"/*.pdb"
    p_list = glob.glob(source)
    # print(p_list, source)
    assert len(p_list) == 1
    p = p_list[0]
    chain_seq, seq = getChainsAndSeq(p)
    
    # print(pdbName, i, len(seq), len(chain_seq))
    decoy_list = []
    with open(f"{pdbFolder}/peptide.txt") as f:
        for line in f:
            pep = line.strip()
            pep_len = len(pep)
            # assert len(pep) == 9
            for c in list(set(chain_seq)):
                if chain_seq.count(c) == pep_len:
                    first_c = chain_seq.find(c)
                    a = list(seq)
                    a[first_c:first_c+pep_len] = pep
                    assert len(set(chain_seq[first_c:first_c+pep_len])) == 1
                    decoy = "".join(a)
                    decoy_list.append(decoy)
    # two branch. one: all become one.
    if len(decoy_list) == 0:
        continue
    opt_pdbName = f"{pdbName}"
    fileLocation = f"{pre}/optimization/decoys/shuffle/{opt_pdbName}.decoys"
    # print("1", pdbName, len(decoy_list), pep_len)
    with open(fileLocation, "w") as out:
        if len(decoy_list) != 1000:
            print("wrong")
            decoy_list = random.choices(decoy_list, k=1000)
        for decoy in decoy_list:
            out.write(decoy+"\n")
    protein_list.append(opt_pdbName)


    target = f"{pre}/database/dompdb/{opt_pdbName}.pdb"
    ## move native pdbs to dompdb
    mycp(p, target)
    ## move native seq to S20_seq
    seq = getSeq(target)
    fileLocation = f"{pre}/database/S20_seq/{opt_pdbName}.seq"
    with open(fileLocation, "w") as out:
        out.write(seq+"\n")
#     # two branch. one: set as 100 per structure.
#     n = int(len(decoy_list) // 100)
#     for i in range(n):
#         opt_pdbName = f"{pdbName}_{i}"
#         fileLocation = f"{pre}/optimization/decoys/shuffle/{opt_pdbName}.decoys"
#         # print("1", pdbName, len(decoy_list), pep_len)
#         with open(fileLocation, "w") as out:
#             for decoy in decoy_list[i*100:(i+1)*100]:
#                 out.write(decoy+"\n")
#         protein_list.append(opt_pdbName)
        
        
#         target = f"{pre}/database/dompdb/{opt_pdbName}.pdb"
#         ## move native pdbs to dompdb
#         mycp(p, target)
#         ## move native seq to S20_seq
#         seq = getSeq(target)
#         fileLocation = f"{pre}/database/S20_seq/{opt_pdbName}.seq"
#         with open(fileLocation, "w") as out:
#             out.write(seq+"\n")
#     fileLocation = f"{pre}/optimization/decoys/shuffle/{pdbName}.decoys"
# #     if len(decoy_list) == 0:
# #         print(pdbName)
#     print("1", pdbName, len(decoy_list), pep_len)
#     with open(fileLocation, "w") as out:
#         for decoy in decoy_list:
#             out.write(decoy+"\n")

In [21]:
len(protein_list)

63

In [22]:
do(f"mkdir -p {pre}/optimization")
do(f"mkdir -p {pre}/phis")

## write protein_list
fileLocation = f"{pre}/optimization/protein_list"
with open(fileLocation, "w") as out:
    for pdbName in protein_list:
        out.write(f"{pdbName}\n")

do(f"cp ~/opt/optimization/phi_list_contact.txt {pre}/optimization/phi_list.txt")

0